In [2]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

connections.connect("default", host="localhost", port="19530")

In [3]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)


In [4]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  



In [5]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)


Status(code=0, message=)

In [8]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
result

data: ['["id: 2998, distance: 0.0, entity: {\'random\': -14.0}", "id: 1730, distance: 0.15077632665634155, entity: {\'random\': -15.0}", "id: 1307, distance: 0.1714766025543213, entity: {\'random\': -19.0}"]', '["id: 2999, distance: 0.0, entity: {\'random\': -12.0}", "id: 2035, distance: 0.13053461909294128, entity: {\'random\': -11.0}", "id: 2694, distance: 0.1518850028514862, entity: {\'random\': -15.0}"]'], cost: 0

In [10]:
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])
result

data: ["{'random': -13.0, 'embeddings': [np.float32(0.37147424), np.float32(0.16600205), np.float32(0.45327848), np.float32(0.09031144), np.float32(0.29048324), np.float32(0.5133159), np.float32(0.48912424), np.float32(0.8724253)], 'pk': 4}", "{'random': -12.0, 'embeddings': [np.float32(0.6582149), np.float32(0.9071849), np.float32(0.32308224), np.float32(0.45927244), np.float32(0.27894968), np.float32(0.9832718), np.float32(0.23531069), np.float32(0.021644324)], 'pk': 5}", "{'random': -13.0, 'embeddings': [np.float32(0.4216679), np.float32(0.06863009), np.float32(0.5040276), np.float32(0.0753118), np.float32(0.3133681), np.float32(0.9328103), np.float32(0.5032293), np.float32(0.88823116)], 'pk': 11}", "{'random': -11.0, 'embeddings': [np.float32(0.992923), np.float32(0.8891927), np.float32(0.98501456), np.float32(0.71963996), np.float32(0.3041172), np.float32(0.8890534), np.float32(0.69525313), np.float32(0.9339917)], 'pk': 13}", "{'random': -13.0, 'embeddings': [np.float32(0.93922585

In [11]:
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])
result

data: ['["id: 2542, distance: 0.19738426804542542, entity: {\'random\': -11.0}", "id: 2471, distance: 0.2613975405693054, entity: {\'random\': -11.0}", "id: 1707, distance: 0.31264767050743103, entity: {\'random\': -11.0}"]', '["id: 2035, distance: 0.13053461909294128, entity: {\'random\': -11.0}", "id: 1805, distance: 0.23580807447433472, entity: {\'random\': -11.0}", "id: 972, distance: 0.2788979113101959, entity: {\'random\': -11.0}"]'], cost: 0